In [ ]:
from google.colab import drive
drive.mount('/content/drive')


# TODO: 레포지토리 경로를 자신의 구글드라이브 환경에 맞게 수정해야 함
%env REPO_ROOT=/content/drive/MyDrive/repos/fairseq
%env TRANSLATION_ROOT=/content/drive/MyDrive/repos/fairseq/examples/translation
%env TEXT=/content/drive/MyDrive/repos/fairseq/examples/translation/iwslt14.tokenized.de-en

In [ ]:
# 이미 클론받았으면 패스
![ ! -d $REPO_ROOT ]&&  git clone https://github.com/pytorch/fairseq.git $REPO_ROOT

In [ ]:
# 레포지토리 루트로 이동
import os
path = os.environ['REPO_ROOT']
os.chdir(path)

# fairseq 설치
!pip install .

In [ ]:
# translation example 디렉토리로 이동
import os
path = os.environ['TRANSLATION_ROOT']
os.chdir(path)

!bash ./prepare-iwslt14.sh

In [ ]:
# preprocessing
![ ! -d $TRANSLATION_ROOT/data-bin ] && fairseq-preprocess --source-lang de --target-lang en \
    --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test \
    --destdir $TRANSLATION_ROOT/data-bin/iwslt14.tokenized.de-en

In [ ]:
# 체크포인트를 저장할 디렉토리 생성
!mkdir -p $TRANSLATION_ROOT/checkpoints/fconv

# train 수행
!CUDA_VISIBLE_DEVICES=0 fairseq-train $TRANSLATION_ROOT/data-bin/iwslt14.tokenized.de-en \
    --optimizer nag --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv_iwslt_de_en --save-dir $TRANSLATION_ROOT/checkpoints/fconv --max-epoch 10